In [7]:
# import necessary libraries

import os
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import binary_fill_holes

In [8]:
# necessary utility functions

def adjust_contrast(img):
    bounds = np.percentile(img, [5, 99.9])
    img = np.clip(img, *bounds)
    return img

def get_bbox(inp):
    coords = np.where(inp != 0)
    minz = np.min(coords[0])
    maxz = np.max(coords[0]) + 1
    minx = np.min(coords[1])
    maxx = np.max(coords[1]) + 1
    miny = np.min(coords[2])
    maxy = np.max(coords[2]) + 1
    return slice(minz, maxz), slice(minx, maxx), slice(miny, maxy)

In [9]:
data_path = 'data/raw/6m'
out_path = 'data/processed/6m'

shapes = []
names = os.listdir(data_path)
for name in names:
    t1 = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(data_path, name, f't1.nii.gz')))
    t2 = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(data_path, name, f't2.nii.gz')))
    seg = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(data_path, name, f'seg.nii.gz')))
    # transpose to axial view
    t1, t2, seg = np.transpose(t1, [1, 2, 0]), np.transpose(t2, [1, 2, 0]), np.transpose(seg, [1, 2, 0])
    t1, t2 = adjust_contrast(t1), adjust_contrast(t2)
    img = np.stack([t1, t2]).astype(np.float32)
    # crop foreground regions
    mask = np.zeros_like(seg).astype(bool)
    for i in range(len(img)):
        mask = mask | (img[i] != 0)
    mask = binary_fill_holes(mask)
    bbox = get_bbox(mask)
    img = img[:, bbox[0], bbox[1], bbox[2]]
    seg = seg[bbox[0], bbox[1], bbox[2]]
    mask = mask[bbox[0], bbox[1], bbox[2]]
    shapes.append(mask.shape)
    # normalization
    for i in range(len(img)):
        img[i][mask] = (img[i][mask] - img[i][mask].min()) / (img[i][mask].max() - img[i][mask].min())
        img[i][mask == 0] = 0
    data = np.concatenate([img, seg[None]])
    np.save(os.path.join(out_path, f'{name}.npy'), data)
print(np.median(shapes, axis=0))

[143. 118. 104.]
